<a href="https://colab.research.google.com/github/somanshu1/Project/blob/main/multi_agentic_rag_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
!pip install fastmcp sentence-transformers faiss-cpu transformers torch google-generativeai duckduckgo-search python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#2 ╔═ Imports & Setup ═════════════════════════════════════════════════════════
import os
import getpass
import logging
import threading
import asyncio
import textwrap

import torch
import uvicorn
import numpy as np
import faiss
import google.generativeai as genai
import nest_asyncio

from transformers import pipeline
from sentence_transformers import SentenceTransformer
from duckduckgo_search import DDGS
from fastmcp import FastMCP, Client
from fastmcp.client.transports import StreamableHttpTransport

# — Logging setup —
logging.basicConfig(
    format="%(asctime)s %(levelname)s: %(message)s",
    level=logging.INFO
)

In [ ]:
#3 ╔═ API Key Configuration ═════════════════════════════════════════════════
# Prompt for API key if not already set in the environment
if "GOOGLE_API_KEY" not in os.environ:
    print("Please enter your Google API key:")
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your GOOGLE_API_KEY: ")

# Configure the Generative AI client
#try:
#    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
#    logging.info("Google Generative AI API configured successfully.")
#except Exception as e:
#    raise RuntimeError(f"Failed to configure Google API: {e}")

Please enter your Google API key:
Enter your GOOGLE_API_KEY: ··········


In [ ]:
#4 ╔═ Model Loading & Constants ══════════════════════════════════════════════
logging.info("Loading models...")
# Use GPU if available
DEVICE = 0 if torch.cuda.is_available() else -1

# Load the summarization and embedding models
SUMMARIZER = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device=DEVICE
)
EMB_MODEL = SentenceTransformer("all-MiniLM-L6-v2")
EMB_DIM = EMB_MODEL.get_sentence_embedding_dimension()

logging.info(f"Models loaded. Embedding dimension: {EMB_DIM}")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#5 ╔═ Document Corpus & Processing ═══════════════════════════════════════════
DOCS = {
    "AI Ethics": """
        Artificial intelligence (AI) ethics is a branch of ethics that studies the moral implications of creating and using artificially intelligent technologies. A primary concern is algorithmic bias, which can arise when AI systems are trained on biased data, leading to unfair or discriminatory outcomes. For example, hiring algorithms might favor candidates from certain demographics if their training data reflects historical hiring biases. Another key area is accountability: who is responsible when an autonomous system, like a self-driving car, makes a mistake that causes harm? Establishing clear lines of responsibility is a complex legal and ethical challenge. Furthermore, the potential for AI to be used for malicious purposes, such as autonomous weapons systems or widespread surveillance, raises profound questions about security and human rights. Ensuring transparency, fairness, and accountability in AI development is crucial for building trust and harnessing its benefits for society.
    """,
    "Photosynthesis Explained": """
        Photosynthesis is the process used by plants, algae, and some bacteria to convert light energy into chemical energy, through a process that converts carbon dioxide and water into glucose (a sugar) and oxygen. This process is essential for life on Earth, as it produces the oxygen we breathe and forms the base of most food chains. The chemical equation for photosynthesis is 6CO2 + 6H2O + Light Energy → C6H12O6 + 6O2. The process occurs in specialized organelles called chloroplasts, which contain the pigment chlorophyll that absorbs light. There are two main stages: the light-dependent reactions, where light energy is captured to create ATP and NADPH, and the Calvin cycle (light-independent reactions), where this energy is used to 'fix' carbon from CO2 into glucose. Factors like light intensity, carbon dioxide concentration, and temperature can all affect the rate of photosynthesis.
    """,
}

logging.info("Processing documents for the router...")
summaries = {}
emb_summaries = {}
for title, text in DOCS.items():
    if len(text.split()) < 60:
        summary = text
    else:
        summary = SUMMARIZER(text, max_length=70, min_length=25, truncation=True)[0]["summary_text"]
    summaries[title] = summary
    emb_summaries[title] = EMB_MODEL.encode(summary, normalize_embeddings=True)

logging.info("Document summaries and embeddings ready.")

In [ ]:
# Cell #6 - Router & RAG Function Setup
# ╔═ Router & RAG Function Setup ═════════════════════════════════════════════

# Import textwrap (missing from original imports)
import textwrap

# --- Build the high-level router with FAISS ---
router_index = faiss.IndexFlatIP(EMB_DIM)
router_titles = list(emb_summaries.keys())
router_vectors = np.vstack([emb_summaries[t] for t in router_titles])
router_index.add(router_vectors)

def route_query(query: str, threshold: float = 0.5) -> str | None:
    """Return the title of the best-matching document, or None if below threshold."""
    qv = EMB_MODEL.encode(query, normalize_embeddings=True).reshape(1, -1)
    scores, idxs = router_index.search(qv, k=1)
    if scores[0, 0] >= threshold:
        return router_titles[idxs[0, 0]]
    return None

logging.info("Router ready.")

# --- Build the low-level document chunk indexes for local RAG ---
doc_chunks: dict[str, tuple[list[str], faiss.IndexFlatIP]] = {}
for title, text in DOCS.items():
    chunks = textwrap.wrap(text, width=1000, break_long_words=False)
    emb_chunks = EMB_MODEL.encode(chunks, normalize_embeddings=True)
    idx = faiss.IndexFlatIP(EMB_DIM)
    idx.add(emb_chunks)
    doc_chunks[title] = (chunks, idx)

def local_rag(query: str, title: str, top_k: int = 3) -> str:
    """Retrieve the most relevant chunks from a specific document."""
    chunks, idx = doc_chunks[title]
    qv = EMB_MODEL.encode(query, normalize_embeddings=True).reshape(1, -1)
    _, idxs = idx.search(qv, min(top_k, len(chunks)))
    return "\n\n---\n\n".join(chunks[i] for i in idxs[0])

logging.info("Local RAG setup complete.")

# Test the setup
print(f"✅ Router configured with {len(router_titles)} documents: {router_titles}")
print(f"✅ Document chunks created: {list(doc_chunks.keys())}")

✅ Router configured with 2 documents: ['AI Ethics', 'Photosynthesis Explained']
✅ Document chunks created: ['AI Ethics', 'Photosynthesis Explained']


In [ ]:
# Cell #7 - MCP Tool Definitions (Fixed)
# ╔═ MCP Tool Definitions ══════════════════════════════════════════════════

mcp = FastMCP("MultiAgentRAG")

# Define the actual functions first
def doc_rag_tool_impl(query: str, title: str) -> dict:
    """Retrieves context from a specific local document given its title."""
    if not title:
        raise ValueError("No document title was provided.")
    context = local_rag(query, title)
    logging.info(f"Retrieved context from document: {title}")
    return {"source": "doc", "title": title, "context": context}

def web_rag_tool_impl(query: str) -> list[dict]:
    """Retrieves context from the web using DuckDuckGo search."""
    logging.info(f"Searching web for: {query}")
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=3):
            results.append({
                "source":  "web",
                "title":   r.get("title", ""),
                "snippet": r.get("body", ""),
                "url":     r.get("href", "")
            })
    return results

# Register them as MCP tools
mcp.tool()(doc_rag_tool_impl)
mcp.tool()(web_rag_tool_impl)

# Create references for direct calls
doc_rag_tool = doc_rag_tool_impl
web_rag_tool = web_rag_tool_impl

logging.info("MCP tools registered.")
print("✅ MCP Server created with multi-agent RAG tools")
print("📌 Tools registered: doc_rag_tool, web_rag_tool")
print("📌 These tools are exposed via MCP and can also be called directly for demo")

✅ MCP Server created with multi-agent RAG tools
📌 Tools registered: doc_rag_tool, web_rag_tool
📌 These tools are exposed via MCP and can also be called directly for demo


In [ ]:
# Cell #8 - Prompt Builder & Demo (Fixed)
# ╔═ Prompt Builder & Client Loop ═══════════════════════════════════════════

# Ensure API key is available
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your GOOGLE_API_KEY: ")

# Re-configure genai
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def build_prompt(task_result: dict | list, query: str) -> str:
    """Formats the output of a RAG tool into a prompt for the LLM."""
    context = ""
    if isinstance(task_result, dict) and task_result.get("source") == "doc":
        context = f"Source Document: {task_result['title']}\n\nContext:\n{task_result['context']}"
    elif isinstance(task_result, list) and task_result and task_result[0].get("source") == "web":
        snippets = [f"Title: {res['title']}\nSnippet: {res['snippet']}\nURL: {res['url']}" for res in task_result if res.get("snippet")]
        context = "Source: Web Search\n\nContext:\n" + "\n\n---\n\n".join(snippets)

    if not context:
        return f"No context was found. Please answer the following user query based on your general knowledge: {query}"

    return f"""Please provide a comprehensive and clear answer to the user's query based **only** on the provided context below. Synthesize the information if necessary.

### Context
{context}

### User Query
{query}

### Answer
"""

# Demo function using exact original logic
def demo_multi_agent_rag(q: str):
    """Demonstrates the complete multi-agent RAG flow."""
    print(f"\n🔍 Processing: '{q}'")

    try:
        # Use exact original routing logic with threshold
        matched_title = route_query(q)

        if matched_title:
            # Use doc_rag_tool - exact original call
            task_result = doc_rag_tool(q, matched_title)
            print(f"📚 Routed to document: {matched_title}")
        else:
            # Use web_rag_tool - exact original call
            logging.info("No local document match. Routing to web search.")
            task_result = web_rag_tool(q)
            print("🌐 Routed to web search")

        # Build prompt using exact original function
        if task_result:
            prompt = build_prompt(task_result, q)
            # Use exact model from original: gemini-1.5-flash-lite
            model = genai.GenerativeModel("gemini-2.0-flash-lite")
            response = model.generate_content(prompt)
            return response.text
        else:
            return "I couldn't find any relevant information to answer your question."

    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return f"Sorry, I encountered an error: {e}"

# ╔═ System Demo ═══════════════════════════════════════════════════════════

print("🤖 Multi-Agent RAG System Ready!")
print("   Documents loaded:", list(DOCS.keys()))
print("   Embedding dimension:", EMB_DIM)
print("   Router threshold: 0.5")
print("   Model: gemini-1.5-flash-lite")
print(f"   API Key configured: {'Yes' if os.environ.get('GOOGLE_API_KEY') else 'No'}\n")

# Apply nest_asyncio for notebook compatibility
nest_asyncio.apply()

# Demo 1: Should match "AI Ethics" document
print("="*80)
print("Demo 1: Testing document routing")
response1 = demo_multi_agent_rag("What is algorithmic bias?")
print(f"\n📝 Response:\n{response1[:500]}...\n")

# Demo 2: Should trigger web search
print("="*80)
print("Demo 2: Testing web search routing")
response2 = demo_multi_agent_rag("What is quantum computing?")
print(f"\n📝 Response:\n{response2[:500]}...\n")

print("="*80)
print("\n✅ Multi-Agent RAG demonstration complete!")

# Quick test function
def test_query(query: str):
    """Test any query through the multi-agent RAG system."""
    return demo_multi_agent_rag(query)

print("\n💡 Test with: test_query('your question')")
print("   Example: test_query('How does photosynthesis work?')")

🤖 Multi-Agent RAG System Ready!
   Documents loaded: ['AI Ethics', 'Photosynthesis Explained']
   Embedding dimension: 384
   Router threshold: 0.5
   Model: gemini-1.5-flash-lite
   API Key configured: Yes

Demo 1: Testing document routing

🔍 Processing: 'What is algorithmic bias?'
📚 Routed to document: AI Ethics


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-lite:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3642.42ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-lite:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 39662.54ms



📝 Response:
Algorithmic bias is a primary concern in AI ethics. It occurs when AI systems are trained on biased data, which leads to unfair or discriminatory outcomes.
...

Demo 2: Testing web search routing

🔍 Processing: 'What is quantum computing?'


/tmp/ipython-input-7-2740516136.py:19: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


🌐 Routed to web search

📝 Response:
Quantum computing is a type of computing that utilizes quantum mechanical phenomena. It is a transformative technology with the potential to solve significant challenges in areas like environment, agriculture, health, energy, climate, and materials.
...


✅ Multi-Agent RAG demonstration complete!

💡 Test with: test_query('your question')
   Example: test_query('How does photosynthesis work?')
